In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

--2024-05-21 07:18:51--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2024-05-21 07:18:51--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6ac9688f7a96963c6d002562eb.dl.dropboxusercontent.com/cd/0/inline/CTThVM4t_ybiBTh1tG-FxgaLqMg14MISSTG1z9SKEwQ7CQCBTHPPu06mrH_T-qCAg2lSnYyrRtV-dLtnLbIRXNZZH0-oy013Ro6o5WTQdilfDzuZqY5ZChv_GoSoYbzaSpc/file# [following]
--2024-05-21 07:18:51--  https://uc6ac9688f7a96963c6d002562eb.dl.dropboxusercontent.com/cd/0/inline/CTThVM4t_ybiBTh1tG-FxgaLqMg14MISSTG1z9SKEwQ7CQCBTHPPu06mrH_T-qCAg2lSnYyrRtV-dLtnLbIRXNZZH0-oy013Ro6o5WTQdilfD

In [3]:
!unzip stacksample.zip?dl=0

Archive:  stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [4]:
!pip install nltk

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
ans_file = r"/content/Answers.csv"
ques_file = r"/content/Questions.csv"
tag_file = r"/content/Tags.csv"

In [7]:
Ans = pd.read_csv(ans_file, encoding='latin-1', on_bad_lines='skip')
Ques = pd.read_csv(ques_file, encoding='latin-1', on_bad_lines='skip')
Tags = pd.read_csv(tag_file, encoding='latin-1',on_bad_lines='skip')

In [8]:
Ques['Title'] = Ques['Title'].str.lower().str.replace('[^\w\s]', '')
Ques['Body'] = Ques['Body'].str.lower().str.replace('[^\w\s]', '')

In [9]:
top_tags = Tags['Tag'].value_counts().head(10).index.tolist()
filtered_tags = Tags[Tags['Tag'].isin(top_tags)]

In [10]:
merged_data = pd.merge(Ques, filtered_tags, on='Id')

In [11]:
mlb = MultiLabelBinarizer()
merged_data['Tag'] = merged_data.groupby('Id')['Tag'].transform(lambda x: list(set(x)))
y = mlb.fit_transform(merged_data['Tag'])


In [12]:
X_train, X_val, y_train, y_val = train_test_split(merged_data['Body'], y, test_size=0.2, random_state=42)


In [13]:
# print(X_train.shape)
# print(X_val.shape)
# print(y_train.shape)
# print(y_val.shape)

## Building Model using LSTM

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Tokenizing the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Padding the sequences
max_sequence_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_sequence_length)

# Building the model
model = Sequential()
model.add(Embedding(5000, 100, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(21, activation='softmax'))

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train_pad, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_val_pad, y_val),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



Epoch 1/2
10335/10335 [==============================] - 3009s 291ms/step - loss: 28.7455 - accuracy: 0.0562 - val_loss: 43.1671 - val_accuracy: 0.0000e+00
Epoch 2/2
10335/10335 [==============================] - 2635s 255ms/step - loss: 61.1926 - accuracy: 0.0931 - val_loss: 78.0683 - val_accuracy: 0.0000e+00


## Evaluating and saving the Model.

In [15]:
model.evaluate(X_val_pad, y_val)
model.save('stack_overflow_tag_predictor.h5')
2

5168/5168 [==============================] - 192s 37ms/step - loss: 78.0683 - accuracy: 0.0000e+00


2